# Menu Database Analysis and Report
### For Analytics, we should use the menu_cstore database
Because column store is optimized to run ad hoc queries on billions of rows without indexes,
combining columnar data on low-cost object storage with multi-threaded query processing
which is perfect for cloud data warehousing/analytics.

In [1]:
import pandas as pd
from IPython.display import display
import mysql.connector

In [2]:
# Functions that used to connect with the cloud database and display results.
def create_connection():
    conn = None
    try:
       conn = mysql.connector.connect(
          user="DB00004175",
          password="tTNlXEirHh)EmwXhtp4koTR",
          host="menu-data-db.mdb0002067.db.skysql.net",
          ssl_ca="skysql_chain.pem",
          port=5001)
       conn.autocommit = False
       return conn
    except mysql.connector.Error as e:
       print(f"Error connecting to MariaDB Platform: {e}")
    return conn

def sql_execute(conn, sql):

    try:
        cur = conn.cursor()
        cur.execute(sql, multi=True)
        conn.commit()
    except mysql.connector.Error as e:
        print(e)

def sql_execute_show(conn, sql):

    try:
        df = pd.read_sql(sql, con = conn)
        display(df)
    except mysql.connector.Error as e:
        print(e)

# Querying Example
- Only call the create_connection() function once
- Use the sql_execute_show() function to display results in python
- Remember to close connection at the end of your session

In [4]:
# Only call once
conn = create_connection()

In [5]:
# sql_execute_show() conn parameter = conn
# sql parameter = the string of you SQL query
sql_execute_show(conn, sql = '''
SELECT year, AVG(calories), AVG(total_fat), AVG(Protein) FROM menu_cstore.nutrition_facts
GROUP BY year
;
''')

,year,AVG(calories),AVG(total_fat),AVG(Protein)
0,2008,477.7172,26.1575,18.9965
1,2010,369.4070,17.5817,12.5405
2,2012,439.7074,32.7203,24.1088
3,2013,370.0542,16.1378,13.4993
4,2014,375.3779,16.7372,13.4871
5,2015,407.4768,18.4459,15.2967
6,2016,378.0453,16.8043,14.0246
7,2017,374.0719,16.9589,13.9133
8,2018,386.7493,17.4970,14.7468


In [16]:
sql_execute_show(conn, sql = '''
SELECT b.r_name, AVG(calories), AVG(total_fat), AVG(Protein)
FROM menu_cstore.nutrition_facts a
LEFT JOIN menu_cstore.restaurant b ON a.r_id = b.r_id
GROUP BY b.r_name
;
''')

,r_name,AVG(calories),AVG(total_fat),AVG(Protein)
0,None,459.4577,24.4788,17.7381
1,7 Eleven,289.9115,14.1728,9.8063
2,Applebee's,535.8140,29.0960,23.8558
3,Arby's,347.4370,15.9087,13.2212
4,Auntie Anne's,275.5607,5.2865,4.4921
...,...,...,...,...
92,Whataburger,434.8670,18.6071,13.5658
93,White Castle,378.2370,13.7935,8.1544
94,Wingstop,220.3913,11.6316,12.3684
95,Yard House,697.2137,40.0058,30.8827


In [17]:
conn.close()
